In [1]:
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup 
import nltk
import re

train = pd.read_csv('labeledTrainData.csv', header=0, delimiter='\t', quoting=3)
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [Errno 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u"you're",
 u"you've",
 u"you'll",
 u"you'd",
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u"she's",
 u'her',
 u'hers',
 u'herself',
 u'it',
 u"it's",
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u"that'll",
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'th

In [2]:
sizeofdata=5000

def review_to_words(raw_review):
    example1 = BeautifulSoup(raw_review, "html.parser")
    example = example1.get_text()
    regex = re.compile('[^a-zA-Z]')
    example_=regex.sub(" ", example)
    words=example_.lower().split()
    stops = set(stopwords.words('english'))
    words = [w for w in words if not w in stops]
    sentence=" ".join(words)
    return sentence

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = sizeofdata)
import numpy as np
test = pd.read_csv("labeledTrainData.csv", header=0, delimiter="\t",quoting=3 )
print test.shape

(25000, 3)


In [4]:
num_reviews = len(test["review"])
clean_test_reviews = []
print "Cleaning the reviews for better analysis\n"
for i in xrange(0,sizeofdata):
    if( (i+1) % 1000 == 0 ):
        print "Review %d of %d\n" % (i+1, num_reviews)
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append(clean_review)

Cleaning the reviews for better analysis

Review 1000 of 25000

Review 2000 of 25000

Review 3000 of 25000

Review 4000 of 25000

Review 5000 of 25000



In [5]:
y=[]
for i in xrange(0,sizeofdata):
    if ((i + 1) % 1000 == 0):
        print "Sentiment %d of %d\n" % (i + 1, num_reviews)
    y.append(test["sentiment"][i])

Sentiment 1000 of 25000

Sentiment 2000 of 25000

Sentiment 3000 of 25000

Sentiment 4000 of 25000

Sentiment 5000 of 25000



In [6]:
test_data_features = vectorizer.fit_transform(clean_test_reviews)
test_data_features = test_data_features.toarray()
y=np.array(y)
from sklearn import cross_validation
x_train, x_test, y_train, y_test=cross_validation.train_test_split(test_data_features,y,test_size=0.20,random_state=0)
print x_train, y_train
print x_test, y_test
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=.00001)
clf.fit(x_train,y_train)
y_pred_class=clf.predict(x_test)
from sklearn import metrics
accuracy=metrics.accuracy_score(y_test, y_pred_class)
accuracy=100*accuracy
print "Accuracy of the classifier =",accuracy,"%"

C:\ProgramData\Anaconda3\envs\python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [1 0 0 ... 1 0 1]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 1 1 1 1 0 1
 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 0
 0 1 1 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1 1 1 0 1 1 0 0
 1 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1
 1 1 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1 0 0
 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 0 0 0 1 1 1
 1 1 1 0 0 0 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 0 0
 1 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0
 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 0 0 1 1 0 1 0 1 1
 1 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 

In [6]:
#END